## Model Representation
### Components of Neural Network
#### A Single Logistic Neuron Unit
- Input features
    - The $x_0$ input is used as "bias unit" and always equal to 1
- Input Wire
- Neuron
- Output Wire
- Output Hypothesis $h_{\theta}(x) = \frac{1}{1+e^{-\theta^Tx}}$

![W4-LOGISTIC-UNIT](Plots/W4-LOGISTIC-UNIT.png)

#### Activation Units
Nodes in the intermediate/hidden layers
- $a^{(j)}_i$: Activation of unit $i$ in layer $j$ (be aware of the notation)
- $\Theta^{(j)}$: matrix of weights controlling function mapping from layer $j$ to layer $j+1$
    - Dimension of $\Theta^{(j)}$: $s_{j+1}*(S_{j}+1)$ if the layer $j$ has $s_j$ units and the layer $j+1$ has $s_{j+1}$ units
    - Each layer has an **additional bias unit** $x_0$/$a_0$

### Forward Propagation - Compute the Hypothesis
In the Neural Net below, we have 3 nodes in the input layers and 3 nodes in the second layer (hidden layer), so the DImension of $\Theta^{(1)}$ is $3*4 = 12$. 

The computation of the second layer (hidden layer) is shown with $a^{(2)}_1,a^{(2)}_2,a^{(2)}_3$.

The final output $h_{\Theta}$ is computed with $a^{(3)}_1$

![W4-FORWARD-PROP](Plots/W4-FORWARD-PROP.png)
- On the right size, it shows the vectorized computation. Note: 
    - Remember to add bias unit (1) to each layer
    - Use $z^{(j)}_k$ to represent the product inside the $g()$ function
        - $z^{(j)} = \Theta^{(j-1)}a^{(j-1)}$
    - The $g()$ in the screenshot stands for the sigmoid function $g(z) = \frac{1}{1+e^{-z}}$

### Examples - From AND/OR to XNOR Operation
XNOR: gives 1 when $x_1=x_2$

An important reference is the curve of the Sigmoid Function
![W4-SIGMOID](Plots/W4-SIGMOID.png)
![W4-NN-OPT](Plots/W4-NN-OPT.png)

### Multiclass Classification
Same as the traditional Logistic Regression, we use **One vs All** method
- The hypothesis output $h_{\theta}(x)$ is a vector instead of a number, and the class with the largest probability will be marked with 1

![W4-NN-MM](Plots/W4-NN-MM.png)